# Preparar el espacio de trabajo en Colab

Para trabajar de forma ordenada, se crearán 3 carpetas en colab:
- *rawData*: en donde estarán los archivos necesarios para trabajar (**input**)
- *results*: en donde se guardarán los archivos generados (**output**)
- *plots*: en donde se guardarán los pdf de las figuras (**output**)

In [ ]:
#@title Crear directorios
system("mkdir /content/rawData", TRUE)
system("mkdir /content/plots", TRUE)
system("mkdir /content/results", TRUE)
system("rm -r /content/sample_data", TRUE) # eliminarmos la carpeta que está por default en colab
system("ls", TRUE) # listamos los elementos del directorio

character(0)

character(0)

character(0)

character(0)

[1] "plots"   "rawData" "results"

En el directorio *rawData* cargaremos los inputs



In [ ]:
#@ title Cargar archivos
setwd("/content/rawData") # establecemos el directorio de trabajo
system("gdown --id 1czZdAibw2GJkvAjg7rzb0B-XakvooqCz") # archivo del diseño experimental
system("gdown --id 1krHb_grVlKn209RybKVwTZcpLIJiUwxn") # archivo con las anotaciones
system("gdown --id 1rozO_ViJd7eriWFojePzJVPB_t_76ceE") # archivo con funciones de R (commonFunctions)
system("gdown --id 108Fjt5z61z6aFd-Db85n3rP_pqa3XA3z") # archivo de metilación
unzip("/content/rawData/methylation.bed.zip") # descomprimimos el archivo de metilación
#system("rm -r /content/rawData/__MACOSX", TRUE)
cat("Lista de archivos disponibles en */rawData* \n")
system("ls", TRUE) # listamos los elementos del directorio

Lista de archivos disponibles en */rawData* 


[1] "commonFunctions.R"   "Design.txt"          "mergedAnnot.csv"    
[4] "methylation.bed"     "methylation.bed.zip"

# Preparar el espacio de trabajo en R




In [ ]:
#@title Instalar y cargar librerías. Definir directorio de trabajo
install.packages(c("data.table","reshape2","ggplot2","vioplot",
"qqman", "vegan"))
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("DSS")  
# Cargar librerías
suppressPackageStartupMessages({
  library("data.table")
  library("reshape2")
  library("ggplot2")
  library("vioplot")
  library("DSS")
  library("qqman")
  library("vegan")
  source(file.path("/content/rawData", "commonFunctions.R"))
})
# Establecer el directorio de trabajo
setwd("/content/results")
getwd()

# Análisis de datos obtenidos por *epiGBS*

## Explorar los datos

In [ ]:
#@title
myData <- f.load.methylation.bed("/content/rawData/methylation.bed")

La tabla de datos (**myData**) contiene 4 columnas con información de las citosinas metiladas:
- **chr**: fragmento o cromosoma (del inglés: chromosome) consenso de las muestras
- **pos**: posición dentro del fragmento 5'->3'
- **context**: nucleótidos alrededor de la citosina metilada (CG,CHH,CHG)
- **samples_called**: número que indica en cuantas muestras se secuenció al menos una vez (1x) cada citosina.

A partir de la 5º columna, se encuentra la información de metilación para cada muestra por duplicado:
  - Shade_40_*methylated*: la terminación **methylated** indica el número de citosinas metiladas
  - Shade_40_*total*: la terminacióón **total** indica el número total de citosinas (suma C metiladas y C no metiladas)

In [ ]:
#@title Datos crudos.
cat("Número de columnas =", dim(myData)[2], "\n")
cat("Número de muestras =", length(grep("_total$",colnames(myData), value = FALSE)), "\n")
cat("Número de filas =", dim(myData)[1],"\n")
cat("\n","\t\t\t\t Vista previa de la tabla:", "\n")
myData[1:5,1:20]

## Control de calidad



Se filtraran citosinas con baja cobertura y aquellas con alta cobertura. Es decir, citosinas que fueron secuenciadas pocas o muchas veces.



**Baja cobertura**: El mínimo es 10 (*10X*), se puede modificar y ser menos estrictos pero esto compromete la confianza en los datos.

**Alta cobertura**: Los valores de cobertura muy altos también son problemáticos ya que se puede detectar citosinas/posiciones diferencialmente metiladas (DMC/DMC) a aquellas C's en que la diferencia se deba a cobertura.

El umbral depende de los investigadores.
Usaremos el valor del cuartil 99,9

In [ ]:
#@ title
#seleccionamos las columnas con la cobertura total
totalCols <- grep("_total$", colnames(myData), value = TRUE)
methCov <- myData[,totalCols]
colnames(methCov) <- gsub("_total$", "", colnames(methCov))
#generamos los bins para observar la distribución de la cobertura de los datos
bins<-c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,1)
# generamos la distribución de la cobertura
quaCov<-quantile(methCov,bins, na.rm = TRUE) 
# set min y max
minCov<-10 # min 10X
maxCov<-quaCov[10] #99.9%
cat("Valor mínimo de cobertura = ", minCov, "\n")
cat("Valor máximo de cobertura = ", maxCov, "\n")
cat("\n \tRango de valores y distribución de la cobertura: \n")
print(quaCov)

**Filtrado**: 
 - Convertimos en NA todas las celdas que no cumplian con los criterios (baja | alta cobertura), 
 - Seleccionamos aquellas citosinas secuenciadas más de 10 veces y menos del max que están *presentes en el **80%** de las muestras*.



Compare esta tabla con la primera.


In [ ]:
#@ title Datos filtrados.
methCov[(methCov < minCov)|(methCov > maxCov)] <- NA
filteredMeth<-f.filter.methylation(myData, methCov) #demora un poquito
write.table(filteredMeth, file = "/content/results/filteredMethylation", sep = '\t', col.names = TRUE, row.names = FALSE, quote = FALSE)
cat("Número de columnas=", dim(filteredMeth)[2], "\n")
cat("Número de muestras=", length(grep("_total$",colnames(filteredMeth), value = FALSE)), "\n")
cat("Número de filas=", dim(filteredMeth)[1],"\n")
cat("\n","\t\t\t\t Vista previa de la tabla:", "\n")
filteredMeth[1:5,1:20]

## Caracterización de la metilación global

Se generarán 3 gráficos para caracterizar la metilación global:
- Histograma: distribución de C's según el % metilación en cada contexto
- Violines: distribución de C's según el % metilación en cada contexto
- Heatmaps: % de metilación para genes, elementos transponibles, repeticiones y regiones no clasificadas.

Se utilizará 
- la matriz de datos filtrados: **filteredMethylation**
- el archivo con las anotaciones para cada cromosoma/fragmento (chr): **mergedAnnot.csv**
- el archivo con el diseño experimental: **Design.txt**

In [ ]:
#@ title Metilación porcentual media para el tratamiento: *Control* y *Shade*.

infileName <- c("/content/results/filteredMethylation")
mePerc <- f.load.methylation.bed(infileName, percentages = TRUE) # see commonFunctions.R
aveData<-f.average.by.group(infileName)
cat("Porcentaje de metilación \n")
aveData[1:5,]

In [ ]:
#@title Histogramas por contexto
f.plot.histogram(mePerc)
#save
pdf(file.path("/content/plots/histograma.pdf"))
f.plot.histogram(mePerc)
invisible(dev.off())

In [ ]:
#@title Violin plots
aveData<-f.average.by.group(infileName)
mePerc <- f.load.methylation.bed(infileName, percentages = TRUE) # see commonFunctions.R
allMeans<-f.plot.violin(aveData, mePerc)
cat("Metilación porcentual \n")
f.plot.violin(aveData, mePerc)
write.csv(round(allMeans, 3), file.path("/content/results/methylationLevelsViolinPlot_means.csv"))  
pdf("/content/plots/methylationLevelsViolinPlot.pdf")
par(oma = c(12, 8, 3, 0), mar = c(0, 0, 0, 0))
f.plot.violin(aveData, mePerc)
invisible(dev.off())

In [ ]:
#@title Heatmap
methPerFeature<-f.plot.heatmap(aveData, mePerc)
cat("Metilación porcentual\n")
pdf("methylationLevelsPerFeature.pdf")
f.plot.heatmap(aveData, mePerc)
invisible(dev.off())
write.csv(methPerFeature, file="/content/results/methylationLevelsPerFeature.csv")

## Efecto del tratamiento de luz en la metilación de C's 

Se realizará 
- un análisis exploratorio mediante un análisis de componentes principales para cada contexto. A partir de la matriz de distancia generada, se realizará un análisis permutacional de la varianza.
- un análisis estadístico mediante DSS para determinar C's diferencialmente metiladas.

In [ ]:
#@title MDS
adonisResult<-f.plot.isoMDS(filteredMeth)
write.csv(adonisResult, file = paste0("/content/results/adonis.csv"), row.names = TRUE)
#pdf(paste0("/content/plots/isoMDS.pdf"), height = 4, width = 10)
#f.plot.isoMDS(filteredMeth)
#invisible(dev.off())

In [ ]:
#@title  Determinación de citosinas diferencialmente metiladas (DMC)
forDSS<-f.format4DSS(filteredMeth)
myBS <- suppressWarnings(makeBSseqData(forDSS, names(forDSS)))
sampleTab <- f.read.sampleTable("Treat") 
allSamples <- gsub("_total$", "", grep("_total$", colnames(filteredMeth), value = TRUE))
sampleTab <- sampleTab[allSamples,]
myFit <- DMLfit.multiFactor(myBS, sampleTab, formula=~Treat)
DMCresult <- DMLtest.multiFactor(myFit, term="Treat")
cat("\n Vista previa del resultado")
DMCresult[1:10,]
write.csv(DMCresult, "/content/results/DMC_analysis.csv",row.names = FALSE)

#@title Aca se puede cortar
DMCresult<-read.csv("/content/results/DMC_analysis.csv")
col2merge<-c('chr','pos')

DMCresult$chrPos <- apply( DMCresult[ , col2merge] , 1 , paste , collapse = "-" )
DMCresult<-DMCresult[order(DMCresult$chrPos),]
filteredMeth$chrPos <- apply( filteredMeth[ ,col2merge] , 1 , paste , collapse = "-" )
filteredMeth<-filteredMeth[order(filteredMeth$chrPos),]
DMCafterFDR<-subset(DMCresult, DMCresult$fdrs <= 0.05)
uniReg<-unique(DMCafterFDR$chr)
chrDMCafterFDR<-matrix(NA, nrow=length(uniReg), ncol = 2)  
for(j in 1:length(uniReg)){
  hits<-sum(DMCafterFDR$chr==uniReg[j])
  colnames(chrDMCafterFDR) <- c("chr","ocurrences")
  chrDMCafterFDR[j,1]<-uniReg[j]
  chrDMCafterFDR[j,2]<-hits
}
chrDMCafterFDR<-data.frame(chrDMCafterFDR)
chrDMCafterFDR <- chrDMCafterFDR[order(-chrDMCafterFDR$ocurrences),]
write.table(DMCafterFDR,"/content/results/DMC_afterFDR.csv",row.names = FALSE, sep="\t", col.names=T, quote=FALSE)


In [ ]:
#@title Manhattan plot
f.plot.manhattan(DMCresult,chrDMCafterFDR)
pdf(paste0("/content/plots/manhattanPlot.pdf"))
f.plot.manhattan(DMCresult,chrDMCafterFDR)
dev.off()

# Guardar en Drive

Se necesita establecer conexión con Drive para obtener el material de trabajo.
Para ello, se utilizará el paquete *googledrive*. 

In [ ]:
#@title Instalar, cargar y configurar el paquete
install.packages(c("googledrive","R.utils","httr"))

#@title Carga los paquetes
suppressPackageStartupMessages({
  library("googledrive")
  library("R.utils")
  library("httr")
})

#@title Autentificar usuario
my_check <- function() {return(TRUE)}
reassignInPackage("is_interactive", pkgName = "httr", my_check) 
options(rlang_interactive=TRUE)

En el siguiente paso, se debe verificar el usuario.

El siguiente código, va a generar dos cosas:
- 1º: Un link para darle acceso al paquete a nuestra cuenta de Drive. Una vez completado los permisos, va a generar un link que hay que **copiar**
- 2º: Un cuadro de texto con el código de autorización: **pegar** el link del primer paso.

In [ ]:
#@title Otorgar permisos
drive_auth(use_oob = TRUE, cache = TRUE)

Una vez que estamos autorizados a editar Drive, vamos a guardar todos los arhivos que están en colab.

In [ ]:
#@title Guardar archivos en el Drive personal
folder2save<-dir('/content', full.names = TRUE)
file2save<-c()
for (f in folder2save){
  file2save<-dir(f, full.names=TRUE)
  for(i in file2save){
    data1 <- drive_upload(i,"~/EpiEvo/02_TP/", overwrite = TRUE) #descarga a cada carpeta personal
  }
}
